# Speeding Up Your Python Codes 1000x

## Introduction

Welcome to "Speeding Up Your Python Codes 1000x"!
In the next 45 minutes, we will learn tricks to unlock python performance and increase a sample code's speed by a factor of 1000x!

## The $n$-body Problem and the Leapfrog Algorithm

The $n$-body problem involves simulating the motion of many bodies interacting with each other through (usually) gravitational forces.
Each body experiences a force from every other body, making the simulation computationally intensive---ideal for exploring optimization techniques.

The leapfrog algorithm is a simple, robust numerical integrator that alternates between updating velocities ("kicks") and positions ("drifts").
It is popular because it conserves energy and momentum well over long simulation periods.

### Gravitational Force/Acceleration Equation

For any two bodies labeled by $i$ and $j$, the direct gravitational force exerted on body $i$ by body $j$ is given by:
\begin{align}
  \mathbf{f}_{ij} = -G m_i m_j\frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^{3/2}},
\end{align}
where:
* $G$ is the gravitational constant.
* $m_i$ and $m_j$ are the masses of the bodies.
* $\mathbf{r}_i$ and $\mathbf{r}_j$ are their position vectors.

Summing the contributions from all other bodies gives the net force (and thus the acceleration) on body $i$:
\begin{align}
  \mathbf{f}_i 
  = \sum_{j\ne i} \mathbf{f}_{ij} 
  = -G m_i \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3},
\end{align}

Given Newton's law $f = m a$, the "acceleration" applied on body $i$ caused by all other bodies is:
\begin{align}
  \mathbf{a}_i 
  = \sum_{j\ne i} \mathbf{a}_{ij} 
  = -G \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3}.
\end{align}

Choosing the right units so $G = 1$.
Here is a pure Python implementation of the gravitational acceleration:

In [ ]:
def acc1(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                axij = - m[j] * (xi - xj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                ayij = - m[j] * (yi - yj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                azij = - m[j] * (zi - zj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

We may try using this function to evaluate the gravitational force between two particles, with with mass $m = 1$, at a distance of $2$.
We expect the "acceleration" be -1/4 in the direction of their seperation.

In [ ]:
m  = [1.0, 1.0]
r0 = [
    (+1.0, 0.0, 0.0),
    (-1.0, 0.0, 0.0),
]
a0ref = [
    (-0.25, 0.0, 0.0),
    (+0.25, 0.0, 0.0),
]

In [ ]:
a0 = acc1(m, r0)

In [ ]:
a0

In [ ]:
assert a0 == a0ref

### Leapfrog Integrator

The leapfrog algorithm is a time-stepping method used to integrate the equations of motion.
It is especially favored for problems like the $n$-body simulation because it is symplectic, meaning it conserves energy and momentum over long periods.
The algorithm updates positions and velocities in a staggered manner:

1. Half-step velocity update (Kick):
   start by updating the velocity by half a time step using the current acceleration $a(t)$:
   \begin{align}
     v\left(t+\frac{1}{2}\Delta t\right) = v(t) + \frac{1}{2}\Delta t\, a(t).
   \end{align}

2. Full-step position update (Drift):
   update the position for a full time step using the half-stepped velocity:
   \begin{align}
     x(t+\Delta t) = x(t) + \Delta t\, v\left(t+\frac{1}{2}\Delta t\right).
   \end{align}

3. Half-step velocity update (Kick):
   finally, update the velocity by another half time step using the new acceleration $a(t+\Delta t)$ computed from the updated positions:
   \begin{align}
     v(t+\Delta t) = v\left(t+\frac{1}{2}\Delta t\right) + \frac{1}{2}\Delta t\, a(t+\Delta t).
   \end{align}

In [ ]:
def leapfrog1(m, r0, v0, dt, acc=acc1):

    n  = len(m)

    # vh = v0 + 0.5 * dt * a0
    a0 = acc(m, r0)
    vh = []
    for i in range(n):
        a0xi, a0yi, a0zi = a0[i]
        v0xi, v0yi, v0zi = v0[i]
        vhxi = v0xi + 0.5 * dt * a0xi
        vhyi = v0yi + 0.5 * dt * a0yi
        vhzi = v0zi + 0.5 * dt * a0zi
        vh.append((vhxi, vhyi, vhzi))

    # r1 = r0 + dt * vh
    r1 = []
    for i in range(n):
        vhxi, vhyi, vhzi = vh[i]
        x0i,  y0i,  z0i  = r0[i]
        x1i = x0i + dt * vhxi
        y1i = y0i + dt * vhyi
        z1i = z0i + dt * vhzi
        r1.append((x1i, y1i, z1i))

    # v1 = vh + 0.5 * dt * a1
    a1 = acc(m, r1)
    v1 = []
    for i in range(n):
        a1xi, a1yi, a1zi = a1[i]
        vhxi, vhyi, vhzi = vh[i]
        v1xi = vhxi + 0.5 * dt * a1xi
        v1yi = vhyi + 0.5 * dt * a1yi
        v1zi = vhzi + 0.5 * dt * a1zi
        v1.append((v1xi, v1yi, v1zi))

    return r1, v1

We may try using this function to evolve the two particles under gravitational force, with some initial velocities.

In [ ]:
v0 = [
    (0.0, +0.5, 0.0),
    (0.0, -0.5, 0.0),
]
v1ref = [
    (-0.024984345739803245, +0.4993750014648409, 0.0),
    (+0.024984345739803245, -0.4993750014648409, 0.0),
]

In [ ]:
r1, v1 = leapfrog1(m, r0, v0, 0.1)

In [ ]:
v1

In [ ]:
assert v1 == v1ref

### Test the Integrator

We test the integrator by evolving the two particle by a time $T = 2\pi$ using $N = 64$ steps.

In [ ]:
from math import pi

N = 64
T = 2 * pi

In [ ]:
dt = T / N
R  = [r0]
V  = [v0]
for _ in range(N):
    r, v = leapfrog1(m, R[-1], V[-1], dt)
    R.append(r)
    V.append(v)

Although our numerical scheme is implemented in pure python, it is still handy to use `numpy` to slice through the data...

In [ ]:
import numpy as np

R = np.array(R)
X = R[:,:,0]
Y = R[:,:,1]

and plot the result...

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, Y, '.-')
plt.gca().set_aspect('equal')

## Benchmarking and Profiling Your Code

Before diving into optimizations, it is essential to understand where our code spends most of its time.
By benchmarking and profiling, we can pinpoint performance bottlenecks and measure improvements after optimization.

### Benchmarking vs. Profiling

* Benchmarking:
  Measures the overall runtime of your code.
  Tools like Python's `timeit` module run your code multiple times to provide an accurate average runtime.
  This helps in comparing the performance before and after optimizations.

* Profiling:
  Provides detailed insights into which parts of your code are consuming the most time.
  For example, `cProfile` generates reports showing function call times and frequencies.
  Note that `cProfile` typically runs the code once, so its focus is on identifying hotspots rather than providing averaged timings.

### Quick Benchmark Example

`timeit` is a module designed for benchmarking by executing code multiple times.
It is excellent for obtaining reliable runtime measurements.

In [ ]:
n  = 1000
m  = np.random.lognormal(size=n).tolist()
r0 = np.random.normal(size=(n, 3)).tolist()
v0 = np.random.normal(size=(n, 3)).tolist()

In [ ]:
%timeit r1, v1 = leapfrog1(m, r0, v0, dt)

It takes about 1.31 second on my laptop to run a single step of leapfrog for $n = 1000$ bodies.
Your mileage may vary.
But this is pretty slow in today's standard.

### Quick Profiling Example

Here is a snippet using cProfile to profile our leapfrog stepper:

In [ ]:
import cProfile

cProfile.run("r1, v1 = leapfrog1(m, r0, v0, dt)")

From cProfile's result, the most used function is `method 'append' of 'list' objects`.
This is not surprising given we've been using for-loop and append, e.g.,
```
    v1 = []
    for i in range(n):
        ...
        v1.append(...)
```
in both `acc1()` and `leapfrog1()`.
This is neither pythonic nor efficient.

## Optimizing 1: Use List Comprehension Over For Loop

Python's list comprehensions is a concise way to create lists by iterating over an iterable and applying an expression---all in a single, compact line.
Internally, they are optimized in `C`, making them generally faster than a standard python for-loop.

In [ ]:
iterable = range(10)

# Standard python for-loop
l1 = []
for x in iterable:
    l1.append(x * 2)

# Using a list comprehension
l2 = [x * 2 for x in iterable]

# Compare results
print(l1)
print(l2)

We may use list comprehension to rewrite our leapfrog algorithm:

In [ ]:
def leapfrog2(m, r0, v0, dt, acc=acc1):

    n  = len(m)

    # vh = v0 + 0.5 * dt * a0
    a0 = acc(m, r0)
    vh = [[v0xi + 0.5 * dt * a0xi 
               for v0xi, a0xi in zip(v0i, a0i)] 
               for v0i,  a0i  in zip(v0,  a0 )]

    # r1 = r0 + dt * vh
    r1 = [[x0i + dt * vhxi
               for x0i, vhxi in zip(r0i, vhi)]
               for r0i, vhi  in zip(r0,  vh )]

    # v1 = vh + 0.5 * dt * a1
    a1 = acc(m, r1)
    v1 = [[vhxi + 0.5 * dt * a1xi
               for vhxi, a1xi in zip(vhi, a1i)]
               for vhi,  a1i, in zip(vh,  a1 )]

    return r1, v1

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt)

In [ ]:
cProfile.run("r1, v1 = leapfrog2(m, r0, v0, dt)")

Depending on your python version, you may see slight performance increase from `timeit` and number of call decreased for `append`.

## Optimizing 2: Reduce Operation Count

Reducing the operation count means cutting down on unnecessary calculations and redundant function calls.
When operations are executed millions of times---as in inner loops or simulations---even small optimizations can yield significant speed improvements.

For example, precomputing constant values or combining multiple arithmetic steps into one reduces repetitive work.
In the context of the $n$-body problem, calculate invariant quantities once outside of critical loops instead of recalculating them every time.
This streamlined approach not only speeds up your code but also help further optimizations like vectorization and JIT compilation.

Recall the benchmark using `leapfrog2()` with `acc1()`.

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc1)

We noticed that the computation of $\mathbf{r}_{ij}^3 = |\mathbf{r}_i - \mathbf{r}_j|^3$ is used in all components of $\mathbf{x}_{ij}$.
Instead of recomputing it, we can simply cache it in a variable `rrr`.
This reduce the benchmark time by about 45% already!!!

In [ ]:
def acc2(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                # "Cache" r^3
                rrr = ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                
                axij = - m[j] * (xi - xj) / rrr
                ayij = - m[j] * (yi - yj) / rrr
                azij = - m[j] * (zi - zj) / rrr

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc2)

But we don't have to stop from this.
The different components of $\mathbf{r}_{ij} = \mathbf{r}_i - \mathbf{r}_j$ can be cached as `dx`, `dy`, `dz`, too.

In [ ]:
def acc3(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                # "Cache" the components of dr = r_ij = ri - rj
                dx = xi - xj
                dy = yi - yj
                dz = zi - zj

                rrr = (dx**2 + dy**2 + dz**2)**(3/2)
                
                axij = - m[j] * dx / rrr
                ayij = - m[j] * dy / rrr
                azij = - m[j] * dz / rrr

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc3)

Similarly, we can cache $-m_j / |\mathbf{r}_{ij}|^3$.

In [ ]:
def acc4(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                dx = xi - xj
                dy = yi - yj
                dz = zi - zj

                rrr = (dx**2 + dy**2 + dz**2)**(3/2)
                f   = - m[j] / rrr # "cache" -m_j / r_{ij}^3

                axi += f * dx
                ayi += f * dy
                azi += f * dz

        a.append((axi, ayi, azi))

    return a

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc4)

Finally, we notice the symmetry that $\mathbf{a}_{ij} = \mathbf{a}_{ji}$.
In principle, we only need to compute $\mathbf{a}_{ij}$ for $j < i$.
However, this requires we pre-allocate a list-of-list.
Just creating list-of-list and use them to keep track of the acceleration actually increase benchmark time.

In [ ]:
def acc5(m, r):
    
    n = len(m)
    a = [[0]*3]*n # create a list-of-list
    for i in range(n):
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                dx = xi - xj
                dy = yi - yj
                dz = zi - zj

                rrr = (dx**2 + dy**2 + dz**2)**(3/2)
                f   = - m[j] / rrr

                # Use the list-of-list to keep track of the acceleration
                a[i][0] += f * dx
                a[i][1] += f * dy
                a[i][2] += f * dz

    return a

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc5)

But once we have the list-of-list, we may change the upper bound of the inner loop to cut the computation to almost half.
This is the fastest code so far.

In [ ]:
def acc6(m, r):
    
    n = len(m)
    a = [[0]*3]*n
    for i in range(n):
        for j in range(i): # adjust the upper bound of the inner loop
            xi, yi, zi = r[i]
            xj, yj, zj = r[j]

            dx = xi - xj
            dy = yi - yj
            dz = zi - zj

            rrr = (dx**2 + dy**2 + dz**2)**(3/2)
            fi  =   m[i] / rrr
            fj  = - m[j] / rrr
            
            a[i][0] += fj * dx
            a[i][1] += fj * dy
            a[i][2] += fj * dz

            # Account the acceleration to the j-th body.
            a[j][0] += fi * dx
            a[j][1] += fi * dy
            a[j][2] += fi * dz

    return a

In [ ]:
%timeit r1, v1 = leapfrog2(m, r0, v0, dt, acc=acc6)